In [1]:
using Pkg
Pkg.activate("../.")
using MuJoCo 
using MeshCat
using MeshCatMechanisms
using DelimitedFiles
using TOML
using Pkg 
using RigidBodyDynamics 
using Rotations: rotation_error, CayleyMap, UnitQuaternion 
using StaticArrays
using ForwardDiff
using LinearAlgebra
using Rotations
using PyPlot 
using QuadrupedBalance

  Activating project at `~/dev/QuadrupedBalance.jl`


LoadError: ArgumentError: Package MuJoCo not found in current path:
- Run `import Pkg; Pkg.add("MuJoCo")` to install the MuJoCo package.


In [2]:
## Mujoco helper functions 
function get_state(mj_data)
    x = zeros(37)
    q = mj_data.qpos
    q̇ = mj_data.qvel
    x[1:4] = q[4:7] 
    x[5:7] = q[1:3]
    x[20:22] = q̇[4:6]
    x[23:25] = q̇[1:3]
    x[8:19] = QuadrupedBalance.mapMotorArrays(q[8:19], QuadrupedBalance.MotorIDs_c, QuadrupedBalance.MotorIDs_rgb)
    x[26:37] = QuadrupedBalance.mapMotorArrays(q̇[7:end], QuadrupedBalance.MotorIDs_c, QuadrupedBalance.MotorIDs_rgb)
    return x 
end 

function get_sensor(mj_data)
    sensor_data = mj_data.sensordata
    acc = sensor_data[1:3]
    ω = sensor_data[4:6]
    encoders = sensor_data[7:18]
    vels = sensor_data[19:30]
    touches = sensor_data[31:end]
    return acc, ω, encoders, vels, touches
end 

function set_state!(muj_data, x)
    muj_data.qpos[1:3] = x[5:7]
    muj_data.qpos[4:7] = x[1:4]
    muj_data.qpos[8:19] = QuadrupedBalance.mapMotorArrays(x[8:19], QuadrupedBalance.MotorIDs_rgb, QuadrupedBalance.MotorIDs_c)  
    muj_data.qvel[1:3] = x[23:25]
    muj_data.qvel[4:6] = x[20:22]
end 

function joint_linear_interpolation(q_start, q_target, rate)
    q_now = q_start * (1-rate) + q_target * rate 
    return q_now 
end 

function joint_pd_control(q, q_des, q_v, Kp, Kd)
    q_diff = q - q_des 
    u = -Kp .* q_diff - Kd .* q_v 
end 

joint_pd_control (generic function with 1 method)

In [3]:
URDFPATH = joinpath(@__DIR__, "..", "src","a1","urdf","a1.urdf")
data = TOML.parsefile("ipopt_eq_point.toml")
x_init = TOML.parsefile("resting.toml")["x_init"]

x_eq = data["x_eq"]
uf = data["u_eq"]
λf = data["λ_eq"];

# Configure the state in the standing pose 
q_stand = x_eq[8:19]
q_stand[2:3] .= q_stand[1]
q_stand[6:7] .= q_stand[5]
q_stand[10:11] .= q_stand[9]

LoadError: UndefVarError: TOML not defined

In [4]:
### Initializing simulator 
A1mech = parse_urdf(URDFPATH, floating=true, remove_fixed_tree_joints=false)
A1 = QuadrupedBalance.UnitreeA1FullBody(A1mech)
mujoco_model = jlModel("../src/a1/urdf/a1.xml")
mujoco_data = jlData(mujoco_model)
h_sim = mujoco_model.opt.timestep
tf = 6.0
times = 0:h_sim:tf
set_state!(mujoco_data, x_init); mj_step(mujoco_model, mujoco_data);
rate_count = 0  

K = readdlm("maximal_lqr_gain.txt", '\t', Float64, '\n')
x_err = zeros(36)
xs = zeros(length(times), 37)
x_errs = zeros(length(times), 36)
us = zeros(length(times), 12);

for i = 1:length(times)
    global rate_count, q_stand
    global acc, ω, encs, vels, touches, err_int
    ## get sensor data 
    acc, ω, encs, vels, touches = get_sensor(mujoco_data)
    encs = QuadrupedBalance.mapMotorArrays(encs, QuadrupedBalance.MotorIDs_c, QuadrupedBalance.MotorIDs_rgb)
    vels = QuadrupedBalance.mapMotorArrays(vels, QuadrupedBalance.MotorIDs_c, QuadrupedBalance.MotorIDs_rgb) 
    
    ## ground truth state 
    x = get_state(mujoco_data) 
    quat_meas = UnitQuaternion(x[1:4])
    p = QuadrupedBalance.fk(encs)
    com_pos = x[5:7] 
    p_RL_foot = quat_meas * p[10:12] + com_pos
    p_FR_foot = quat_meas * p[1:3] + com_pos
    p_support = p_FR_foot[1:2] - p_RL_foot[1:2]
    P_project = p_support * p_support' / (p_support' * p_support)

    ## calc error and control  
    if (rate_count < 3000)
        rate_count+=1 
        rate = rate_count > 2000 ? 1  : (rate_count / 2000)
        q_des = joint_linear_interpolation(x_init[8:19], q_stand, rate)
        u = joint_pd_control(x[8:19], q_des, x[26:end], 100, 0)
    else 
        quat_meas = UnitQuaternion(x[1:4])
        quat_eq = UnitQuaternion(x_init[1:4])
        θ_err = rotation_error(quat_meas, quat_eq, Rotations.CayleyMap()) 

        # Position error 
        eq_point = (p_FR_foot[1:2] + p_RL_foot[1:2])/2.0
        x_err[4:5] =  quat_eq[1:2,1:2]' * (com_pos[1:2] - eq_point) #- (quat_meas' * ([0.01, 0.0, 0.0]))[1:2]
        x_err[6] = (x[7] - x_eq[7])
        
        x_err[1:3] = θ_err 
        x_err[7:18] = encs - x_eq[8:19]
        x_err[19:21] = ω 
        x_err[22:23] = quat_eq[1:2,1:2]' * x[23:24] 
        x_err[24] = x[25] 
        x_err[25:36] = vels
        
        ## Attitude and joint errors 
#         x_err[1:3] = θ_err
#         x_err[4:15] = encs - x_eq[8:19]
#         x_err[16:18] = ω 
#         x_err[19:30] = vels
         
        u = -K*x_err + uf
        us[i,:] = u  
    end 

    xs[i,:] = x 
    x_errs[i,:] = x_err 

    ## setting control and simulate a step
    mujoco_data.ctrl[:] = QuadrupedBalance.mapMotorArrays(u, QuadrupedBalance.MotorIDs_rgb, QuadrupedBalance.MotorIDs_c)
    mj_step(mujoco_model, mujoco_data)
end 

LoadError: UndefVarError: parse_urdf not defined

In [5]:
vis = Visualizer() 
cur_path = pwd()
cd(joinpath(@__DIR__,"..","src", "a1", "urdf"))
mvis = MechanismVisualizer(A1mech, URDFVisuals(URDFPATH), vis)
cd(cur_path)

LoadError: UndefVarError: Visualizer not defined

In [6]:
q_anim = [xs[i,1:19] for i in 1:length(times)-1]
animation = Animation(mvis, times[1:50:end-1], q_anim[1:50:end])
setanimation!(mvis, animation);

LoadError: UndefVarError: times not defined

In [7]:
plot(x_errs[3001:end,4:6])

LoadError: UndefVarError: x_errs not defined

In [8]:
render(vis)

LoadError: UndefVarError: render not defined